In [22]:
import os
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Criar pasta para armazenar os dados
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

# Carregar o dataset
dataset = pd.read_excel("Server_1_Training_Sets/datasets/cpu_usage_1-13.xlsx")
#dataset = pd.read_excel("Server_1_Training_Sets/datasets/ram_usage_1-13.xlsx")
# dataset = pd.read_excel("Server_1_Training_Sets/network_usage_1-13.xlsx")

# Preparar os dados
X_columns = dataset.columns[:len(dataset.columns)-1]
X = np.array(dataset[X_columns]).reshape(-len(X_columns), len(X_columns))
y = np.array(dataset["y"]).reshape(-1, 1)

X_data = torch.tensor(X, dtype=torch.float32)
y_data = torch.tensor(y, dtype=torch.float32)

# Dividir os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=2024, shuffle=True)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).clone().detach()
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).clone().detach()

X_val_tensor = torch.tensor(X_val, dtype=torch.float32).clone().detach()
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).clone().detach()

# Salvar os dados em disco
torch.save((X_train, y_train), os.path.join(data_dir, "train_data.pt"))
torch.save((X_val, y_val), os.path.join(data_dir, "val_data.pt"))

print(f"Dados de treinamento e validação salvos em {data_dir}.")

Dados de treinamento e validação salvos em ./data.


/tmp/ipykernel_176427/271170338.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32).clone().detach()
/tmp/ipykernel_176427/271170338.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32).clone().detach()
/tmp/ipykernel_176427/271170338.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(X_val, dtype=torch.float32).clone().detach()
/tmp/ipykernel_176427/271170338.py:32: UserWarning: To copy co

In [23]:
# Criar uma classe customizada para o dataset
class CustomDataset(Dataset):
    def __init__(self, data_path):
        self.X, self.y = torch.load(data_path)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Carregar os dados com DataLoader
train_dataset = CustomDataset(os.path.join(data_dir, "train_data.pt"))
val_dataset = CustomDataset(os.path.join(data_dir, "val_data.pt"))

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=True)

/tmp/ipykernel_176427/3803462038.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.X, self.y = torch.load(data_path)


In [24]:
# Checagem do train test split
# Verifica se uma das amostras separadas para o X_train e y_train é compatível com o conjunto de dados original "dataset"

dataset_sample = list(X_train[0]) + list(y_train[0])

# Checagem do CustomDataset
# Verifica se uma das amostras separadas para o PyTorch CustomDataset é compatível com o conjunto de dados original "dataset"

xysample = list(train_dataset.__getitem__(0)[0]) + list(train_dataset.__getitem__(0)[1])

# Checagem do DataLoader
# Verifica se uma das amostras separadas para o Dataloader é compatível com o conjunto de dados original "dataset"
next_train_loader_sample = next(iter(train_loader))
dataloader_xysample = next_train_loader_sample[0][0].tolist() + next_train_loader_sample[1][0].tolist()

assert len(dataset[(dataset[dataset.columns] == xysample).all(axis=1)].index) >= 1
assert len(dataset[(dataset[dataset.columns] == xysample).all(axis=1)].index) >= 1
assert len(dataset[(dataset[dataset.columns] == dataset_sample).all(axis=1)].index) >= 1

In [25]:
import torch
import torch.nn as nn
import numpy as np
from torch.optim import Adam
import os

'''class SimpleLSTM(nn.Module):
    def __init__(self):
        super(SimpleLSTM, self).__init__()
        
        self.hidden_size = 256
        self.num_layers = 3
        
        self.lstm = nn.LSTM(input_size=40, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(2 * self.hidden_size, 1)
        #self.linear = nn.Linear(self.hidden_size, 1)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        #x = self.linear(x[:, -1, :])  # Apenas a última saída do LSTM
        return x'''
    
class SimpleLSTM(nn.Module):
    def __init__(self):
        super(SimpleLSTM, self).__init__()
        
        self.hidden_size = 256
        self.num_layers = 3
        self.lstm = nn.LSTM(input_size=40, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, 1)  # Ajustado para LSTM unidirecional

        #self.linear1 = nn.Linear(40, 30)
        #self.relu1 = nn.ReLU()
        #self.linear2 = nn.Linear(30, 20)
        #self.relu2 = nn.ReLU()
        #self.linear3 = nn.Linear(20, 10)
        #self.relu3 = nn.ReLU()
        #self.linear4 = nn.Linear(10, 1)
        #self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        # Garantir que a entrada tenha 3 dimensões
        x, _ = self.lstm(x)  # x: (batch_size, seq_length, hidden_size)
        x = self.linear(x)  # Apenas a última saída da sequência

        #x = self.linear1(x)
        #x = self.relu1(x)
        #x = self.linear2(x)
        #x = self.relu2(x)
        #x = self.linear3(x)
        #x = self.relu3(x)
        #x = self.linear4(x)
        #x = self.sigmoid(x)

        return x

def train(model, optimizer, loss_fn, device, n_epochs, max_consecutive_increases):
    global train_rmse_history
    global val_rmse_history

    best_rmse = float('inf')
    best_epoch = 0
    consecutive_increases = 0

    train_rmse_history = []
    val_rmse_history = []

    for epoch in range(n_epochs):
        model.train()  # Certifique-se de que o modelo está em modo de treinamento
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * X_batch.size(0)  # Acumula a loss total ponderada pelo tamanho do batch
        
        # Calcular RMSE de treino médio para a época
        train_rmse = np.sqrt(train_loss / len(train_loader.dataset))

        # Avaliação no conjunto de validação
        val_loss = 0.0
        with torch.no_grad():  # Desabilita o cálculo de gradientes
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                val_loss += loss.item() * X_batch.size(0)  # Acumula a loss total ponderada pelo tamanho do batch
        
        # Calcular RMSE de validação médio para a época
        val_rmse = np.sqrt(val_loss / len(val_loader.dataset))
        
        # Salvar histórico de RMSE
        train_rmse_history.append(train_rmse)
        val_rmse_history.append(val_rmse)
        
        # Printar os resultados da época
        print(f"Epoch {epoch+1}/{n_epochs}: train RMSE {train_rmse:.4f}, val RMSE {val_rmse:.4f}")
        
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            best_epoch = epoch
            checkpoint_path = f"models/model_best_rmse.pt"
            torch.save(model, checkpoint_path)
            print(f"Modelo salvo com RMSE: {best_rmse} na época {epoch}")
        
        with open("models/performance.txt", "w") as f:
            f.write(f"Best Model Val RMSE: {best_rmse}\nModel Best Val Epoch: {best_epoch}\nModel Train RMSE History: {train_rmse_history}\nModel Val RMSE History: {val_rmse_history}")

        if len(val_rmse_history) > 1 and val_rmse > val_rmse_history[-2]:
            consecutive_increases += 1
        else:
            consecutive_increases = 0
        
        if consecutive_increases >= max_consecutive_increases:
            print(f"Early stopping at epoch {epoch} due to consecutive increases in RMSE.")
            break
        
    print(f"Melhor RMSE encontrado: {best_rmse} na época {best_epoch}")

In [26]:
if not os.path.exists('models'):
    os.makedirs('models')

model = SimpleLSTM()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

optimizer = Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
n_epochs = 100
max_consecutive_increases = 10

train(model, optimizer, loss_fn, device, n_epochs, max_consecutive_increases)

Epoch 1/100: train RMSE 0.1619, val RMSE 0.0451
Modelo salvo com RMSE: 0.045121141626541086 na época 0
Epoch 2/100: train RMSE 0.0438, val RMSE 0.0434
Modelo salvo com RMSE: 0.043407717576865394 na época 1
Epoch 3/100: train RMSE 0.0418, val RMSE 0.0411
Modelo salvo com RMSE: 0.04114491229387079 na época 2
Epoch 4/100: train RMSE 0.0394, val RMSE 0.0383
Modelo salvo com RMSE: 0.03826570986259798 na época 3
Epoch 5/100: train RMSE 0.0360, val RMSE 0.0345
Modelo salvo com RMSE: 0.03452984269871815 na época 4
Epoch 6/100: train RMSE 0.0324, val RMSE 0.0313
Modelo salvo com RMSE: 0.03133420731334361 na época 5
Epoch 7/100: train RMSE 0.0306, val RMSE 0.0301
Modelo salvo com RMSE: 0.03013293817790016 na época 6
Epoch 8/100: train RMSE 0.0300, val RMSE 0.0304
Epoch 9/100: train RMSE 0.0300, val RMSE 0.0302
Epoch 10/100: train RMSE 0.0300, val RMSE 0.0303
Epoch 11/100: train RMSE 0.0300, val RMSE 0.0298
Modelo salvo com RMSE: 0.029787855746308774 na época 10
Epoch 12/100: train RMSE 0.0299, v

# Training Evolution

In [ ]:
import matplotlib.pyplot as plt

metric_name = "CPU Usage"
#metric_name = "RAM Usage"
#metric_name = "Network Usage"
#metric_name = "Resource Usage"
model_name = "LSTM"

plt.plot(train_rmse_history)
plt.title(f"{metric_name} - {model_name} - Train RMSE Evolution")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.legend()
plt.grid()
#plt.show()
plt.savefig(f"plots/{metric_name} {model_name}.png", dpi=300)